🕺 This ipynb file is draft for aggregation models for free events about IT and programming: webinars, hackathons, broadcasts and courses.

# TODO:
- Extract file imports in one file
- Make specific module for yandex features and common handler class
- For hand-written content: update of events may be not safe
- Write parser for pages of yandex events: div class:events-info
    - Handlers for required_event_keys:
        + 'url'
        + 'title'
        + 'date'
        + 'time'
        + 'location': div class="events-info__location"
        + 'online status'
    - Handlers for optional_event_keys
        + 'registration_url': Registration link 
        + 'price'
        + 'tags'
        + 'language'
        + 'speakers'
        + 'themes'
        - 'type'
    - Check if all required values for the event are found

# Strategy
Shortly: first we check the main pages, then event pages

- Load `pages.json` - information about start wepbages: `name`, `url`, `func`
- Load `events.json` - information about current events:
    - `title`,
    - `url` (description),
    - `registration url` (optional, often the same as description url),
    - `date`
    - `time`
    - `location`
    - `online status`
    - `price`
    - `[tags]`
    - `language` (can be checked by lang of theme)
    - `speakers and themes (optional)`
    - `type (optional)`
- Check that events on the main pages are displayed in `events.json`
- If the event is not in the file, we need to add basic information about it on the second stage
- Past events are transfered to `past_events.json`
- On the next stage if any information is missing, we need to supplement it by going to specific url and parse the page
- For registration url we will add utm information

In [1]:
import json

# custom packages
import files
import soups
import urls
import handlers

Узнаем, какие из стартовых страниц, перечисленных в `files.pages`, размечены. Такие страницы содержат все поля, перечисленные `files.fields`.

In [2]:
pages_checked = files.pages_checked()
pages_checked

['Яндекс']

Для обработки полей, описывающих события, логична следующая последовательность:
- Поля, считываемые со стартовой страницы
- Поля, расположенные на странице мероприятия
- Поля, получаемые из анализа предобработанных полей

In [3]:
files.fields_order('Яндекс')    

{'start': ['event_url', 'title', 'date'],
 'event': ['time',
  'location',
  'online_status',
  'price',
  'reg_url',
  'themes',
  'speakers',
  'speakers_companies',
  'organizers',
  'prize'],
 'analysis': ['language', 'tags', 'event_type']}

In [4]:
# def start_page_processing(name):
#     start_url = files.pages[name]['start_url']
#     print(f'Parsing page {name} at {}')
#     soup = soups.get(start_url)
#     event_cards = soup.body.find_all('div', {'class':'event-card'})
#     for event_card in event_cards:
#         date_string = event_card.find('div', {'class':'event-card__date'}).text
#         event_date = yandex.date_converter(date_string)
#         event_url = event_card.a['href']
#         event_url = urls.add_home(event_url, start_url)
#         print(event_url)
#         events[event_url] = {'start':name,
#                              'event_url':event_url,
#                              'title':event_card.a['title'],
#                              'date':event_date}

In [5]:
name = 'Яндекс'

start_fields = files.fields_order(name)['start']
start_url = files.pages[name]['start_url']
start = dict()

for field in start_fields:
    start[field] = files.get_content(name, start_url, field)

Яндекс event_url
event-card__title
Яндекс title
event-card__title
Яндекс date
event-card__date


In [10]:
for i, event_url in enumerate(start['event_url']):
    event_url = handlers.event_url(event_url, name)
    files.events[event_url] = {}
    files.events[event_url]['start_page'] = name
    for field in start_fields:
        if field != 'event_url':
            files.events[event_url][field] = start[field][i]

In [11]:
files.events

{'https://events.yandex.ru/events/yasubbotnik/4-jul-2020': {'start_page': 'Яндекс',
  'title': 'Я.Субботник по разработке интерфейсов. Онлайн, темы выбираете вы!',
  'date': 'сб, 4 июля'},
 'https://cloud.yandex.ru/events/145': {'start_page': 'Яндекс',
  'title': 'about:cloud – бессерверные технологии и IoT',
  'date': 'чт, 2 июля'},
 'https://cloud.yandex.ru/events/143': {'start_page': 'Яндекс',
  'title': 'Вебинар. Как работает сеть в Облаке',
  'date': 'вт, 30 июня'},
 'https://events.yandex.ru/events/zen-meetup/26-06-2020': {'start_page': 'Яндекс',
  'title': 'Дзен-митап: видеорекомендации. Онлайн',
  'date': 'пт, 26 июня'},
 'https://events.yandex.ru/events/bugs-busters-25-06-2020': {'start_page': 'Яндекс',
  'title': 'BugsBusters',
  'date': 'чт, 25 июня'},
 'https://cloud.yandex.ru/events/138': {'start_page': 'Яндекс',
  'title': 'Вебинар. Начало пути в GameDev',
  'date': 'чт, 25 июня'},
 'https://cloud.yandex.ru/events/140': {'start_page': 'Яндекс',
  'title': 'Вебинар. Обзор 

In [14]:
with open('events.json', 'w', encoding='utf-8') as events_file:
    json.dump(files.events, events_file, ensure_ascii=False)

# Let's write Event class and handlers for all fields

In [ ]:
import handlers

all_fields = set(fields.keys())
required   = {key for key in fields if fields[key]['required']}
optional   = {key for key in fields if not fields[key]['required']}


class Event:
    def __init__(self, url):
        self.data = events[url]
        self.data = self.update(url)
        
    def update(self, url):
        today = current_date.isoformat()
        available = set(self.data.keys())
        analysis = {field for field in pages['Яндекс'] if pages['Яндекс'][field].get('selectors') == 'analysis'}
        if self.data['date'] >= today:
            #if any(required - available):
            fields = all_fields - available - analysis
            for field in fields:
                self.data[field] = get_content(url, field)
            self.data['tags'] = list(self.tags())

    
    def tags(self):
        tags = {}
        title = self.data['title']
        event_url = self.data['event_url']
        themes = self.data['themes'].copy()
        themes.append(title)
        themes.append(event_url)
        text = ''.join(themes)
        for key in tags:
            for tag in tags[key]:
                if tag.lower() in text.lower():
                    self.data[tags].add(tag)
        return tags

    
    def display(self):
        print(self.data)

In [ ]:
event = Event('https://cloud.yandex.ru/events/145')

In [ ]:
print(event.data)

# Let's process HTML and deploy the current version of the project 💃

In [ ]:
soup = BeautifulSoup()
section = soup.new_tag('section')
section.string = "new test"
soup.append(section)
str(soup)

In [ ]:
content = str(soup) # for test

from pathlib import Path
path_to_html_template = Path("../matyushkin.github.io/11ty/_includes/events.njk")

with open(path_to_html_template, 'wt', encoding='utf-8') as html_template:
    html_template.write(content)

I use [11ty](https://www.11ty.dev/) processing for manipulations with templates, headers and footers of html files on my [web site](https://matyushkin.github.io/). So the simplest way to include and deploy new files at GitHub pages is a Jupyter cell with simple bash code. I use `;` operator because Jupyter returns to current pwd on a new line.

In [ ]:
!cd ../matyushkin.github.io/11ty/; npx eleventy --passthroughall --output=..

# Let's see quickly what we have now
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

If all is OK, it's only one line for deployment at GitHub pages:

In [ ]:
! cd ../matyushkin.github.io/; git add . ; git commit -m "Events project files are updated"; git push origin master